In [ ]:
# !pip install fastapi uvicorn


In [ ]:
# !pip install fastapi uvicorn nest-asyncio -q

In [1]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np
import threading
import asyncio
import nest_asyncio

app = FastAPI()

# Chargement des modèles
model = joblib.load("modele_prediction_maladie.pkl")
scaler = joblib.load("scaler.pkl")
ohe = joblib.load("onehot_encoder.pkl")
mlb = joblib.load("symptom_encoder.pkl")

# Modèle de données
class Patient(BaseModel):
    age: int
    sexe: str
    fumeur: str
    tension: float
    temperature: float
    symptomes: list[str]

@app.post("/predict")
def predict(patient: Patient):
    if len(patient.symptomes) < 3:
        return {"error": "Au moins 3 symptômes requis."}

    input_num = scaler.transform([[patient.age, patient.tension, patient.temperature]])
    input_cat = ohe.transform([[patient.sexe, patient.fumeur]])
    input_sympt = mlb.transform([patient.symptomes])
    final_input = np.concatenate([input_num, input_cat, input_sympt], axis=1)

    proba = model.predict_proba(final_input)[0]
    classe = model.classes_[np.argmax(proba)]
    return {"maladie": classe, "probabilite": float(np.max(proba))}


def run_server():
    config = uvicorn.Config(
        app,
        host="0.0.0.0",
        port=8006,
        loop="asyncio",
        reload=True
    )
    server = uvicorn.Server(config)
    
    # Création d'une nouvelle event loop pour ce thread
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    
    try:
        loop.run_until_complete(server.serve())
    finally:
        loop.close()





: 

In [1]:
# Lancement du serveur dans un thread

if __name__ == "__main__":
    # Lancement du serveur dans un thread démon
    server_thread = threading.Thread(
        target=run_server,
        daemon=True,
        name="FastAPI-Server"
    )
    server_thread.start()
    
    print("Serveur FastAPI démarré sur http://0.0.0.0:8006")
    
    try:
        # Maintenir le thread principal actif
        while True:
            # Vous pourriez ajouter ici d'autres logiques de supervision
            server_thread.join(0.1)  # Timeout court pour permettre l'interruption
            if not server_thread.is_alive():
                print("Le serveur a arrêté de fonctionner")
                break
    except KeyboardInterrupt:
        print("\nArrêt demandé par l'utilisateur")
    finally:
        print("Nettoyage terminé")

NameError: name 'threading' is not defined